In [1]:
from DatasetCreation import DatasetCreation

In [2]:
dsc = DatasetCreation('/media/philipp/DATA/2018_tamsweg/')

In [3]:
paths = dsc.find_files('/media/philipp/DATA/2018_tamsweg/', ['ortho', 'dsm', 'dtm', 'slope', 'ground_truth'])

In [4]:
len(paths['ortho'])

10679

In [5]:
paths['slope']

['/media/philipp/DATA/2018_tamsweg/slope/tile_slope_185427.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_125673.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_186650.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_154072.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_203879.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_200990.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_146979.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_157409.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_139510.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_154793.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_206698.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_197879.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_186269.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_187095.tif',
 '/media/philipp/DATA/2018_tamsweg/slope/tile_slope_191961.tif',
 '/media/philipp/DATA/201

In [6]:
import numpy as np

data_types = {'ortho':np.uint8, 'dsm':np.float16, 'dtm':np.float16, 'ground_truth':np.uint8}

In [18]:
data_dic = {'ortho':{'dtype':np.uint8, 'dim':4}, \
              'dsm':{'dtype':np.float16, 'dim':1}, \
              'dtm':{'dtype':np.float16, 'dim':1}, \
              'ground_truth':{'dtype':np.uint8}, 'dim':1}

In [19]:
list(data_types.keys())[0]

'ortho'

In [20]:
data_types['ortho']['dim']

4

In [8]:
'ground_truth' in paths.keys()

True

In [9]:
x = dsc.x(paths, 0, 100, data_types)

(100, 512, 512, 4)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)


In [13]:
x['dsm'].shape

(371, 256, 256, 1)

In [22]:
10300//1000

10

In [23]:
10300%1000

300

In [32]:
ds_size = 10300
block_size = 1000
end = 0
for i in range(ds_size // block_size):
    end = block_size * (i+1)
    print(end-block_size, end)
print(end, end + ds_size % block_size)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 10300


In [30]:
ds_size = 300
block_size = 1000
print(0, ds_size)

0 300


In [3]:
dsc.create_hdf5('/media/philipp/DATA/dataset/dataset_df_0.h5', tile_size=256, dataset_size=40000)

KeyError: 'dim'

In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
from osgeo import gdalconst

from glob import glob
import logging
import os

from DatasetCreation import DatasetCreation

In [2]:
data_dic = {'ortho':{'dtype':np.uint8, 'dim':4}, \
              'dsm':{'dtype':np.float16, 'dim':1}, \
              'dtm':{'dtype':np.float16, 'dim':1}, \
              'slope':{'dtype':np.float16, 'dim':1}, \
              'ground_truth':{'dtype':np.uint8, 'dim':1}}

data_types = ['ortho', 'dsm', 'dtm', 'slope', 'ground_truth']

tile_size = 256

dataset_size = 40000

block_size = 1000

In [3]:
dsc = DatasetCreation('/media/philipp/DATA/2018_tamsweg/')

In [4]:
paths = dsc.find_files('/media/philipp/DATA/2018_tamsweg/', ['ortho', 'dsm', 'dtm', 'slope', 'ground_truth'])

In [6]:
"""
path_hdf5 = '/media/philipp/DATA/dataset/dataset_df_0.h5'

hdf5_ds = h5py.File(path_hdf5, 'a')

for data_type in data_types:
    x = hdf5_ds.create_dataset(data_type, \
    (dataset_size, tile_size, tile_size, data_dic[data_type]['dim']), \
    dtype=data_dic[data_type]['dtype'])
hdf5_ds.close()
"""

"\npath_hdf5 = '/media/philipp/DATA/dataset/dataset_df_0.h5'\n\nhdf5_ds = h5py.File(path_hdf5, 'a')\n\nfor data_type in data_types:\n    x = hdf5_ds.create_dataset(data_type,     (dataset_size, tile_size, tile_size, data_dic[data_type]['dim']),     dtype=data_dic[data_type]['dtype'])\nhdf5_ds.close()\n"

In [7]:
path_hdf5 = '/media/philipp/DATA/dataset/dataset_df_0.h5'
hdf5_ds = h5py.File(path_hdf5, 'a')

In [8]:
hdf5_ds['dsm']

<HDF5 dataset "dsm": shape (40000, 256, 256, 1), type "<f2">

In [9]:
ds_size = len(paths[list(paths.keys())[0]])
print(ds_size)

10679


In [11]:
if ds_size > block_size:
    counter = 0
    for i in range(ds_size // block_size):

        print(block_size*i, block_size*(i+1))

        data = dsc.prepare_dataset(paths, block_size*i, block_size*(i+1), data_dic)

        for data_type in data_types:
            # set dataset
            x = hdf5_ds[data_type]
            # assign data
            x[counter:counter+data[data_type].shape[0],:,:,:] = data[data_type]
            
        counter += data[data_type].shape[0]
        print(counter)

    print(block_size*(i+1), block_size*(i+1) + ds_size % block_size)

0 1000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
3662
1000 2000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
7345
2000 3000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
11043
3000 4000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
14701
4000 5000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
18353
5000 6000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
22033
6000 7000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
25702
7000 8000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
29366
8000 9000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 

In [14]:
data = dsc.prepare_dataset(paths, block_size*(i+1), ds_size, data_dic)

for data_type in data_types:
    # set dataset
    x = hdf5_ds[data_type]
    # assign data
    x[counter:counter+data[data_type].shape[0],:,:,:] = data[data_type]

counter += data[data_type].shape[0]
print(counter)

(679, 512, 512, 4)
(679, 512, 512, 1)
(679, 512, 512, 1)
(679, 512, 512, 1)
(679, 512, 512, 1)
39280


In [24]:
hdf5_ds['ortho'][10]

array([[[ 66,  83,  74, 167],
        [ 57,  72,  64, 157],
        [ 48,  63,  55, 147],
        ...,
        [ 41,  49,  36, 127],
        [ 40,  47,  36, 123],
        [ 46,  54,  44, 131]],

       [[ 56,  73,  66, 158],
        [ 45,  59,  51, 144],
        [ 45,  57,  51, 142],
        ...,
        [ 45,  53,  38, 135],
        [ 51,  60,  47, 144],
        [ 61,  74,  63, 159]],

       [[ 42,  52,  48, 138],
        [ 39,  49,  43, 133],
        [ 37,  44,  38, 127],
        ...,
        [ 43,  49,  31, 131],
        [ 51,  61,  44, 144],
        [ 61,  73,  60, 158]],

       ...,

       [[ 73,  80,  54, 161],
        [ 76,  83,  57, 164],
        [ 74,  80,  55, 160],
        ...,
        [ 61,  69,  43, 163],
        [ 68,  79,  54, 171],
        [ 69,  80,  54, 172]],

       [[ 77,  84,  57, 165],
        [ 77,  84,  56, 164],
        [ 85,  95,  70, 172],
        ...,
        [ 72,  82,  58, 174],
        [ 78,  91,  66, 179],
        [ 77,  89,  64, 179]],

       [[ 81

In [25]:
hdf5_ds.close()